In [ ]:
%cd M:/Rafael/Documentos/GitHub/TCC
import pandas as pd 
import matplotlib.pyplot as plt
import json
import numpy as np
from datetime import datetime
from preprocessa import recupera_arquivos_data
from vector import recupera_arquivos_token
from tqdm import tqdm

In [ ]:
# Importa bibliotecas e carrega modelo Word2Vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
wv = KeyedVectors.load("artigos.wv", mmap='r')

In [ ]:
# Exemplo de word-vector 
print(*list(wv['política']), sep='\t')

In [ ]:
def plota_similaridade(palavra, n):
    ''' Representa através de gráfico de barras o grau de similaridade de uma palavra com as N mais próximas '''
    lista = wv.most_similar(positive=[palavra], negative=[], topn=n)
    df = pd.DataFrame(lista, columns=['palavra', 'grau similaridade'])
    df = df.set_index('palavra')
    df['grau similaridade'] = (df['grau similaridade']*100).astype(int)
    
    df.plot(kind="barh",title=palavra,  legend=False, width=0.95)
    plt.axis("off")
    for i, (p, pr) in enumerate(zip(df.index, df["grau similaridade"])):
        plt.text(s=p, x=1, y=i, color="w", verticalalignment="center", size=14)
        plt.text(s=str(pr)+"%", x=pr-12, y=i, color="w",
                 verticalalignment="center", horizontalalignment="left", size=12)
    plt.rcParams["figure.figsize"] = (4,3)
    plt.xticks([])
    plt.yticks([])
    plt.gca().invert_yaxis()
    plt.savefig("sim_" + palavra +".png")

In [ ]:
def plota_similaridade(palavra, n):
    ''' Representa através de gráfico de barras o grau de similaridade de uma palavra com as N mais próximas '''
    lista = wv.most_similar(positive=[palavra], negative=[], topn=n)
    df = pd.DataFrame(lista, columns=['palavra', 'grau similaridade'])
    df = df.set_index('palavra')
    df['grau similaridade'] = (df['grau similaridade']*100).astype(int)
    
    df.plot(kind="barh",title=palavra,  legend=False, width=0.95)
    plt.axis("off")
    for i, (p, pr) in enumerate(zip(df.index, df["grau similaridade"])):
        plt.text(s=p, x=1, y=i, color="w", verticalalignment="center", size=14)
        plt.text(s=str(pr)+"%", x=pr-12, y=i, color="w",
                 verticalalignment="center", horizontalalignment="left", size=12)
    plt.rcParams["figure.figsize"] = (4,3)
    plt.xticks([])
    plt.yticks([])
    plt.gca().invert_yaxis()
    plt.savefig("sim_" + palavra +".png")
    
for palavra in ['lula', 'dilma', 'dória', 'marina', 'ciro', 'bolsonaro', 'eleição', 'trump',
                'esquerda', 'direita', 'cloroquina', 'covid', 'pandemia', 'china', 'vacina']:
    plota_similaridade(palavra, 5)


In [ ]:
# Carrega o dataset
df = pd.read_csv('dataset_final.csv', encoding='utf-8',sep=';', low_memory=False)

In [ ]:
df

In [ ]:
df.groupby(df['portal']).size()

In [ ]:
# Criando DataFrame com documentos de 'Esquerda' e removendo as colunas 'portal' e 'posição'
df_e = df.loc[df['posicao'] == 'Esquerda']
df_e = df_e.drop(['portal', 'posicao'], axis=1)
df_e               

In [ ]:
# Criando DataFrame com documentos de 'Direita' e removendo as colunas 'portal' e 'posição'
df_d = df.loc[df['posicao'] == 'Direita']
df_d = df_d.drop(['portal', 'posicao'], axis=1)
df_d

In [ ]:
# Convertendo DataFrames para listas de artigos, contendo cada um uma lista de palavras
dados_d = df_d.values.tolist()
dados_e = df_e.values.tolist()

In [ ]:
len(dados_e)

In [ ]:
len(dados_d)

In [ ]:
def monta_palavras_vec(dados):
    ''' Retorna um NumPy Arrray de documentos X features. Cada documento possui 300 dimensões, ou features p/ ML '''
    artigos_vec = np.zeros((len(dados), 300))
    tamanho = len(dados)
    print(artigos_vec.shape)
    # Monta vetores para todos os documentos do dataset
    for i, artigo in tqdm(enumerate(dados), total=tamanho, desc='Processando', delay=1):
        for palavra in artigo:
            # Interrompe ao encontrar NaN (fim do artigo)
            if type(palavra) != str:
                break
            artigos_vec[i] += wv[palavra] 
        artigos_vec[i] /= len(artigo)   
    return artigos_vec
    
artigos_vec_d = monta_palavras_vec(dados_d)        
artigos_vec_e = monta_palavras_vec(dados_e)     

In [ ]:
from sklearn.utils import resample

# Downsampling da classe majoritária (Esquerda)
downsampled = resample(artigos_vec_e, 
                       replace=True,     
                       n_samples=artigos_vec_d.shape[0],    
                       random_state=123) 

In [ ]:
# Concatenando coluna com zeros para rotular a classe 'E' ou 'Esquerda' 
artigos_vec_e = downsampled
classe_e = np.zeros((artigos_vec_e.shape[0], 1), dtype=np.int8)
artigos_vec_e = np.concatenate((artigos_vec_e, classe_e), axis=1)

In [ ]:
artigos_vec_e.shape

In [ ]:
# Concatenando coluna com digito 1 para rotular a classe 'D' ou 'Direita'
classe_d = np.ones((artigos_vec_d.shape[0], 1), dtype=np.int8)
artigos_vec_d = np.concatenate((artigos_vec_d, classe_d), axis=1)

In [ ]:
artigos_vec_d.shape

In [ ]:
# Concatena as duas classes em uma só matriz
artigos_vec = np.concatenate((artigos_vec_d, artigos_vec_e))
artigos_vec.shape

In [ ]:
# Demonstrando que não existem 'missing values' no dataset
print(np.where(np.isnan(artigos_vec)))

In [ ]:
artigos_vec.shape[0]

In [ ]:
df = pd.DataFrame(artigos_vec)
df

In [ ]:
# Importa bibliotecas do scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


# Train/Test Split
tam_teste = 0.25
X = artigos_vec[:,:300]
y = artigos_vec[:,-1:].ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=tam_teste)

In [ ]:
# library
import matplotlib.pyplot as plt

# create data
tamanho_grupos=[round((1-tam_teste)*100), round(tam_teste*100)]

label_treino = f'     Dados Teste ({tamanho_grupos[1]}%)\n\n     {y_test.shape[0]:_} registros     '.replace("_", ".")
label_teste = f'Dados Treino ({tamanho_grupos[0]}%) \n\n {y_train.shape[0]:_} registros      '.replace("_", ".")
labels = label_teste, label_treino        

# Create a pieplot
plt.pie(tamanho_grupos, explode=(0, 0.06), labels=labels, autopct='',
        shadow=True, startangle=0)

# add a circle at the center to transform it in a donut chart
my_circle=plt.Circle( (0,0), 0.6, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)

plt.show()

In [ ]:
def relatorio_classificacao(nome_modelo, resultado_modelo, y_test):
    print(f'\nAlgoritmo - {nome_modelo}\n\nRelatório de Classificação\n\n', 
          classification_report(y_test, resultado_modelo),'\n\nMatriz de Confusão\n\n', 
          confusion_matrix(resultado_modelo, y_test))   


In [ ]:
# Criando modelo e treinando com os dados de treino
clr = LogisticRegression(max_iter=300)
clr.fit(X_train, y_train)
# Fazendo a predição nos dados de teste
resultado_clr = clr.predict(X_test)
# Principais métricas de performance e Matriz de Confusão
relatorio_classificacao("Regressão Logística", resultado_clr, y_test)

In [ ]:
# Criando modelo e treinando com os dados de treino
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
# Fazendo a predição nos dados de teste
resultado_dtc = dtc.predict(X_test)
# Principais métricas de performance e Matriz de Confusão
relatorio_classificacao("Árvore de Decisão", resultado_dtc, y_test)

In [ ]:
# Criando modelo e treinando com os dados de treino
rf=RandomForestClassifier(n_jobs=6, n_estimators=400)
rf.fit(X_train,y_train)
# Fazendo a predição nos dados de teste
resultado_rf = rf.predict(X_test)
# Principais métricas de performance e Matriz de Confusão
relatorio_classificacao("Random Forest", resultado_rf, y_test)

In [ ]:
# Criando modelo e treinando com os dados de treino
ada = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=123)
ada.fit(X_train, y_train)
# Fazendo a predição nos dados de teste
resultado_ada = ada.predict(X_test)
# Principais métricas de performance e Matriz de Confusão
relatorio_classificacao("AdaBoost", resultado_ada, y_test)

In [ ]:
# Criando modelo e treinando com os dados de treino
knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                           metric_params=None, n_jobs=6, n_neighbors=5, p=2,
                           weights='uniform')
knn.fit(X_train, y_train)                           
# Fazendo a predição nos dados de teste
resultado_knn = knn.predict(X_test)
# Principais métricas de performance e Matriz de Confusão
relatorio_classificacao("K-Nearest Neighbors", resultado_knn, y_test)